In [97]:
import sys
import os
import math
import pandas as pd
import numpy as np
import json

# Python visualisation library
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# Another much simpler viz library
import seaborn as sns

# Adding root of git repo to path so we can import pyfireanalytics
sys.path.append('..')

# Display formatting options
pd.set_option('display.max_columns', 1000)

# Load Data

In [98]:
data = pd.read_csv('~/desktop/movielens/UDATA.csv', encoding="latin-1")
item = pd.read_csv('~/desktop/movielens/UITEM.csv', encoding="latin-1")
genre = pd.read_csv('~/desktop/movielens/UGENRE.csv', encoding="latin-1")
user = pd.read_csv('~/desktop/movielens/UUSER.csv', encoding="latin-1")
#"ISO-8859-1"

# Data Cleaning

In [99]:
# rename itemid to movieid to join data and item together
data.rename(columns={'itemid':'movieid'}, inplace=True)

#join
df = pd.merge(data, item, on='movieid', how='outer')
df = pd.merge(df, user, on='userid', how='outer')

#change unix timestamp to readable date
df['reviewtimestamp'] = pd.to_datetime(df['timestamp'],unit='s')

#Split reviewtimestamp into year month day
df['reviewyear'] = df['reviewtimestamp'].dt.year
df['reviewmonth'] = df['reviewtimestamp'].dt.month
df['reviewday'] = df['reviewtimestamp'].dt.day


# Collaborative Filtering Model

In [100]:
#Step 1: Find the top similar users using Centred Cosine Similarity

#centred cosine
ccs = data.groupby(['userid'], as_index=False, sort=False).mean()
ccs = ccs.rename(columns={"rating": "mean_rating"})
data = pd.merge(data, ccs, on='userid', how='left', sort="False")

data['rating'] = data['rating'].astype(float)
data = data.copy()
data['new_rating'] = data["rating"]-data['mean_rating']

data

,userid,movieid_x,rating,timestamp_x,movieid_y,mean_rating,timestamp_y,new_rating
0,1,61,4.0,878542420,136.500000,3.610294,8.773916e+08,0.389706
1,1,189,3.0,888732928,136.500000,3.610294,8.773916e+08,-0.610294
2,1,33,4.0,878542699,136.500000,3.610294,8.773916e+08,0.389706
3,1,160,4.0,875072547,136.500000,3.610294,8.773916e+08,0.389706
4,1,20,4.0,887431883,136.500000,3.610294,8.773916e+08,0.389706
5,1,202,5.0,875072442,136.500000,3.610294,8.773916e+08,1.389706
6,1,171,5.0,889751711,136.500000,3.610294,8.773916e+08,1.389706
7,1,265,4.0,878542441,136.500000,3.610294,8.773916e+08,0.389706
8,1,155,2.0,878542201,136.500000,3.610294,8.773916e+08,-1.610294
9,1,117,3.0,874965739,136.500000,3.610294,8.773916e+08,-0.610294


In [106]:
data2=data.copy()
umr = pd.DataFrame({"user": data2['userid'], "movie":data2['movieid_x'], "rating":data2['new_rating']})
umr2 = umr.pivot_table(index='user', columns='movie', values='rating').fillna(0)

In [124]:
allusers = umr2.values
user1 = allusers[0]
denominator1 = np.sqrt(sum([np.square(x) for x in user1]))

cosinesimilarity = [(1,1)]
i=1
for user in allusers[1:]:
        numerator = [x*y for x,y in zip(user1, user)]
        denominator2 = np.sqrt(sum([np.square(x) for x in user]))
        costheta = sum(numerator) / (denominator1 * denominator2)
        cosinesimilarity.append((umr2.index[i], costheta))
        i += 1

cosinesimilarity.sort(key=lambda x: x[1], reverse=True)

similar10users = cosinesimilarity[0:10]

top10users = pd.DataFrame()
for user in similar10users:
    top10users = top10users.append(umr2.loc[user[0]])
top10users['costheta'] = [user[1] for user in similar10users]

all_values = top10users.values

top10users

1         2         3         4         5         6         7  \
1    1.389706 -0.610294  0.389706 -0.610294 -0.610294  1.389706  0.389706   
773 -0.279503 -0.279503  0.000000  0.000000  0.000000 -0.279503 -1.279503   
868  1.048077 -0.951923  0.000000  0.000000  0.000000  0.000000  2.048077   
592  0.186111  0.000000  0.186111  0.186111  0.000000  0.000000  1.186111   
880  0.573370 -0.426630 -2.426630  0.573370 -0.426630  0.000000 -0.426630   
429 -0.393720 -0.393720 -1.393720  0.606280  0.000000  0.000000 -1.393720   
276  1.534749  0.534749 -0.465251  0.534749 -0.465251  0.000000  1.534749   
916  0.634069 -0.365931 -0.365931  0.634069 -0.365931  0.000000  0.634069   
222  0.950904 -0.049096  0.000000 -0.049096  0.000000  0.000000  1.950904   
457 -0.025271  0.000000  0.000000 -0.025271  0.000000  0.000000 -0.025271   

            8         9        10        11        12        13        14  \
1   -2.610294  1.389706 -0.610294 -1.610294  1.389706  1.389706  1.389706   
773  0.000000  0.000000  0.000000 -1.279503 -0.279503  0.720497  1.720497   
868  0.000000  0.000000  0.000000  0.000000  2.048077  0.000000  0.000000   
592  1.186111  1.186111  0.000000  1.186111  1.186111  1.186111  1.186111   
880  0.573370  0.000000  0.000000  0.573370  1.573370  0.000000  0.000000   
429 -0.393720  0.000000  0.000000  0.606280  1.606280  0.000000  0.000000   
276  0.534749  1.534749  0.000000  1.534749  1.534749  0.000000  0.534749   
916  0.000000  1.634069  0.000000  0.634069  0.634069  0.000000  1.634069   
222 -2.049096  1.950904  0.000000  1.950904  1.950904  0.000000  0.000000   
457  0.974729  0.974729  0.000000 -0.025271  0.974729 -1.025271 -0.025271   

           15        16        17        18        19        20        21  \
1    1.389706  1.389706 -0.610294  0.389706  1.389706  0.389706 -2.610294   
773  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
868  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
592  1.186111  0.000000  0.000000  0.000000  0.000000  0.186111  0.000000   
880  0.000000  0.000000 -0.426630  0.000000  0.000000  0.000000 -1.426630   
429  1.606280  0.000000  0.000000  0.000000  0.000000  0.000000 -1.393720   
276  0.000000  0.000000  0.534749  0.000000  0.000000  0.000000 -0.465251   
916  0.000000  0.000000  0.634069  0.000000  0.000000  0.000000  0.000000   
222 -0.049096  0.000000 -1.049096  0.000000  0.000000  0.000000  0.000000   
457 -0.025271  0.000000  0.000000  0.000000  0.000000  0.974729  0.000000   

           22        23        24        25        26        27        28  \
1    0.389706  0.389706 -0.610294  0.389706 -0.610294 -1.610294  0.389706   
773  0.000000  1.720497 -0.279503  0.000000  0.000000 -2.279503  0.000000   
868  0.000000  2.048077 -0.951923  0.000000  0.000000  0.000000  0.000000   
592  1.186111  1.186111  0.186111  0.000000  0.000000  0.000000  0.186111   
880  0.573370  1.573370 -0.426630  0.573370  0.000000 -0.426630  1.573370   
429  1.606280  0.606280 -0.393720  0.606280 -0.393720  0.000000 -0.393720   
276  1.534749  1.534749  0.534749  0.534749  0.000000 -0.465251  0.534749   
916  0.634069  0.634069 -1.365931  0.000000  0.000000  0.000000  0.634069   
222  1.950904  0.000000 -0.049096 -0.049096 -0.049096  0.000000  1.950904   
457  0.974729  0.000000  0.000000 -0.025271  0.000000 -0.025271  0.974729   

           29        30        31        32        33        34        35  \
1   -2.610294 -0.610294 -0.610294  1.389706  0.389706 -1.610294 -2.610294   
773 -1.279503  0.000000  0.000000  0.720497  0.000000  0.000000  0.000000   
868  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
592  0.000000  0.000000  0.000000  1.186111  0.000000  0.000000  0.000000   
880 -1.426630  0.000000  0.573370  0.000000 -0.426630  0.000000  0.000000   
429  0.000000  0.000000 -0.393720  0.606280  0.000000  0.000000  0.000000   
276 -0.465251  0.000000  0.534749  0.000000  0.534749 -1.465251  0.000000   
916 

In [127]:
#Step 2: Predict the user ratings on an item based on other users

denominator = sum([x[1] for x in similar10users])

inx = 0
values= []
for x in top10users.loc[1]:
    totalsum=0
    if x ==0.0:
        for v in range(1,10):
            totalsum += all_values[v-1][inx]*all_values[v-1][1682]
        top10users.loc[1][inx+1] = totalsum/denominator
    inx += 1
    
top10users

1         2         3         4         5         6         7  \
1    1.389706 -0.610294  0.389706 -0.610294 -0.610294  1.389706  0.389706   
773 -0.279503 -0.279503  0.000000  0.000000  0.000000 -0.279503 -1.279503   
868  1.048077 -0.951923  0.000000  0.000000  0.000000  0.000000  2.048077   
592  0.186111  0.000000  0.186111  0.186111  0.000000  0.000000  1.186111   
880  0.573370 -0.426630 -2.426630  0.573370 -0.426630  0.000000 -0.426630   
429 -0.393720 -0.393720 -1.393720  0.606280  0.000000  0.000000 -1.393720   
276  1.534749  0.534749 -0.465251  0.534749 -0.465251  0.000000  1.534749   
916  0.634069 -0.365931 -0.365931  0.634069 -0.365931  0.000000  0.634069   
222  0.950904 -0.049096  0.000000 -0.049096  0.000000  0.000000  1.950904   
457 -0.025271  0.000000  0.000000 -0.025271  0.000000  0.000000 -0.025271   

            8         9        10        11        12        13        14  \
1   -2.610294  1.389706 -0.610294 -1.610294  1.389706  1.389706  1.389706   
773  0.000000  0.000000  0.000000 -1.279503 -0.279503  0.720497  1.720497   
868  0.000000  0.000000  0.000000  0.000000  2.048077  0.000000  0.000000   
592  1.186111  1.186111  0.000000  1.186111  1.186111  1.186111  1.186111   
880  0.573370  0.000000  0.000000  0.573370  1.573370  0.000000  0.000000   
429 -0.393720  0.000000  0.000000  0.606280  1.606280  0.000000  0.000000   
276  0.534749  1.534749  0.000000  1.534749  1.534749  0.000000  0.534749   
916  0.000000  1.634069  0.000000  0.634069  0.634069  0.000000  1.634069   
222 -2.049096  1.950904  0.000000  1.950904  1.950904  0.000000  0.000000   
457  0.974729  0.974729  0.000000 -0.025271  0.974729 -1.025271 -0.025271   

           15        16        17        18        19        20        21  \
1    1.389706  1.389706 -0.610294  0.389706  1.389706  0.389706 -2.610294   
773  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
868  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
592  1.186111  0.000000  0.000000  0.000000  0.000000  0.186111  0.000000   
880  0.000000  0.000000 -0.426630  0.000000  0.000000  0.000000 -1.426630   
429  1.606280  0.000000  0.000000  0.000000  0.000000  0.000000 -1.393720   
276  0.000000  0.000000  0.534749  0.000000  0.000000  0.000000 -0.465251   
916  0.000000  0.000000  0.634069  0.000000  0.000000  0.000000  0.000000   
222 -0.049096  0.000000 -1.049096  0.000000  0.000000  0.000000  0.000000   
457 -0.025271  0.000000  0.000000  0.000000  0.000000  0.974729  0.000000   

           22        23        24        25        26        27        28  \
1    0.389706  0.389706 -0.610294  0.389706 -0.610294 -1.610294  0.389706   
773  0.000000  1.720497 -0.279503  0.000000  0.000000 -2.279503  0.000000   
868  0.000000  2.048077 -0.951923  0.000000  0.000000  0.000000  0.000000   
592  1.186111  1.186111  0.186111  0.000000  0.000000  0.000000  0.186111   
880  0.573370  1.573370 -0.426630  0.573370  0.000000 -0.426630  1.573370   
429  1.606280  0.606280 -0.393720  0.606280 -0.393720  0.000000 -0.393720   
276  1.534749  1.534749  0.534749  0.534749  0.000000 -0.465251  0.534749   
916  0.634069  0.634069 -1.365931  0.000000  0.000000  0.000000  0.634069   
222  1.950904  0.000000 -0.049096 -0.049096 -0.049096  0.000000  1.950904   
457  0.974729  0.000000  0.000000 -0.025271  0.000000 -0.025271  0.974729   

           29        30        31        32        33        34        35  \
1   -2.610294 -0.610294 -0.610294  1.389706  0.389706 -1.610294 -2.610294   
773 -1.279503  0.000000  0.000000  0.720497  0.000000  0.000000  0.000000   
868  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
592  0.000000  0.000000  0.000000  1.186111  0.000000  0.000000  0.000000   
880 -1.426630  0.000000  0.573370  0.000000 -0.426630  0.000000  0.000000   
429  0.000000  0.000000 -0.393720  0.606280  0.000000  0.000000  0.000000   
276 -0.465251  0.000000  0.534749  0.000000  0.534749 -1.465251  0.000000   
916 

In [135]:
#Step 3: Recommend the items which have higher predicted value

top10rec = []

for inx,x in enumerate(top10users.loc[1].values):
    if x > 0.0:
        top10rec.append((inx,x))
        
top10rec.sort(key=lambda x: x[1], reverse=True)

top10rec[:10]

[(0, 1.3897058823529411),
 (5, 1.3897058823529411),
 (8, 1.3897058823529411),
 (11, 1.3897058823529411),
 (12, 1.3897058823529411),
 (13, 1.3897058823529411),
 (14, 1.3897058823529411),
 (15, 1.3897058823529411),
 (18, 1.3897058823529411),
 (31, 1.3897058823529411)]